In [1]:
import pandas as pd
from google.colab import files
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

In [2]:
import pandas as pd
from google.colab import files

# List of file paths
file_paths = [
    "PublicDebt_Indicators.csv",
    "Economic_Indicators.csv",
    "Social_Indicators.csv",
    "Environmental_Indicators.csv",
    "Statistical_Indicators.csv"
]

# Initialize an empty list to store transformed DataFrames
dfs = []

# Process each CSV file
for file in file_paths:
    try:
        # Load the CSV file
        df = pd.read_csv(f"/content/{file}", index_col=False)

        # Identify year columns dynamically (ignoring first four ID columns)
        year_columns = df.columns[4:]

        # Melt the DataFrame without creating extra columns
        df_melted = pd.melt(
            df,
            id_vars=["Country Name", "Country Code", "Series Name", "Series Code"],
            value_vars=year_columns,  # Use dynamically detected year columns
            var_name="Year",
            value_name="Value"
        )

        # Special handling for "PublicDebt_Indicators.csv" (Keep quarters like 2022Q1)
        if "PublicDebt" in file:
            df_melted["Year"] = df_melted["Year"].astype(str).str.extract(r'(\d{4}Q[1-4])')

        else:  # Extract only the year for other files
            df_melted["Year"] = df_melted["Year"].astype(str).str.extract(r'(\d{4})')

        # Append the transformed DataFrame to the list
        dfs.append(df_melted)
        print(f"Processed: {file}  Rows: {len(df_melted)}")

    except Exception as e:
        print(f"Error processing {file}: {e}")

# Combine all reshaped DataFrames into one
final_df = pd.concat(dfs, ignore_index=True)

# Display a preview of the final combined dataset
print("\nFinal Combined Data Preview:")
print(final_df.head(10))

# Save the final combined dataset
final_df.to_csv("Combined_Database.csv", index=False)

# Download the final dataset
files.download("Combined_Database.csv")


Processed: PublicDebt_Indicators.csv  Rows: 58905
Processed: Economic_Indicators.csv  Rows: 36723
Processed: Social_Indicators.csv  Rows: 53481
Processed: Environmental_Indicators.csv  Rows: 19167
Processed: Statistical_Indicators.csv  Rows: 13035

Final Combined Data Preview:
  Country Name Country Code  \
0      Albania          ALB   
1      Albania          ALB   
2      Albania          ALB   
3      Albania          ALB   
4      Albania          ALB   
5      Albania          ALB   
6      Albania          ALB   
7      Albania          ALB   
8      Albania          ALB   
9      Albania          ALB   

                                         Series Name           Series Code  \
0  Gross PSD, General Gov., All maturities, All i...  DP.DOD.DECD.CR.GG.Z1   
1  Gross PSD, General Gov., All maturities, All i...  DP.DOD.DECX.CR.GG.Z1   
2  Gross PSD, General Gov., All maturities, All i...  DP.DOD.DECT.CR.GG.Z1   
3  Gross PSD, General Gov., All maturities, All i...  DP.DOD.DECF.CR

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
# Load the dataset
file_path = "Combined_Database.csv"
df = pd.read_csv(file_path)

# Display basic info
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181311 entries, 0 to 181310
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Country Name  181230 non-null  object
 1   Country Code  181176 non-null  object
 2   Series Name   181176 non-null  object
 3   Series Code   181176 non-null  object
 4   Year          169530 non-null  object
 5   Value         169410 non-null  object
dtypes: object(6)
memory usage: 8.3+ MB
None
  Country Name Country Code  \
0      Albania          ALB   
1      Albania          ALB   
2      Albania          ALB   
3      Albania          ALB   
4      Albania          ALB   

                                         Series Name           Series Code  \
0  Gross PSD, General Gov., All maturities, All i...  DP.DOD.DECD.CR.GG.Z1   
1  Gross PSD, General Gov., All maturities, All i...  DP.DOD.DECX.CR.GG.Z1   
2  Gross PSD, General Gov., All maturities, All i...  DP.DOD.DECT.CR.GG.Z1   
3 

<ipython-input-4-cf25695d55ca>:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [5]:
# Display missing values before processing
print("\nMissing Values Before Handling:\n", df.isnull().sum())


Missing Values Before Handling:
 Country Name       81
Country Code      135
Series Name       135
Series Code       135
Year            11781
Value           11901
dtype: int64


In [6]:
# Convert "Value" column to float
df["Value"] = pd.to_numeric(df["Value"], errors="coerce")

# Verify conversion
print("\nUpdated Data Types:\n", df.dtypes)


Updated Data Types:
 Country Name     object
Country Code     object
Series Name      object
Series Code      object
Year             object
Value           float64
dtype: object


In [7]:
# Drop rows where both "Value" and "Year" are missing
df = df.dropna(subset=["Value", "Year"], how="all")

print("\nDeleted rows where both Value and Year were missing.")


Deleted rows where both Value and Year were missing.


In [8]:
# Get the latest number of rows after deletion
latest_row_count = len(df)

print(f"\nLatest number of rows after deletion: {latest_row_count}")


Latest number of rows after deletion: 169530


In [9]:
# Count missing values before filling
missing_values_before = df[["Country Name", "Country Code"]].isnull().sum()

# Fill missing "Country Name" using "Country Code"
df["Country Name"] = df.groupby("Country Code")["Country Name"].ffill().bfill()

# Fill missing "Country Code" using "Country Name"
df["Country Code"] = df.groupby("Country Name")["Country Code"].ffill().bfill()

# Count missing values after filling
missing_values_after = df[["Country Name", "Country Code"]].isnull().sum()

# Calculate the number of filled values
filled_values = missing_values_before - missing_values_after

# Display the number of missing values filled
print("\nNumber of missing values filled:")
print(filled_values)


Number of missing values filled:
Country Name     67
Country Code    115
dtype: int64


In [10]:
# Store missing values count before filling
missing_values_before = df["Value"].isnull().sum()

# Check if there are any missing values before proceeding
if missing_values_before == 0:
    print("\nNo missing values found in 'Value' column. Nothing to fill.")
else:
    # Fill missing "Value" using median within the same Series Name & Series Code
    df["Value"] = df.groupby(["Series Name", "Series Code"])["Value"].transform("median")

    # Store missing values count after filling
    missing_values_after = df["Value"].isnull().sum()

    # Calculate the number of filled values
    filled_values = missing_values_before - missing_values_after

    print(f"\nNumber of missing values filled in 'Value' column: {filled_values}")


Number of missing values filled in 'Value' column: 86597


In [11]:
print("\nMissing numerical values in 'Value' column AFTER filling:")
print(df["Value"].isnull().sum())


Missing numerical values in 'Value' column AFTER filling:
3312


In [12]:
df = df.dropna(subset=["Value"])

In [13]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Standardize the "Value" column
df["Value"] = scaler.fit_transform(df[["Value"]])

# Display standardized data
print("\nStandardized 'Value' Column:\n", df["Value"].head())


Standardized 'Value' Column:
 0   -0.235872
1   -0.235872
2   -0.235872
3   -0.235872
4   -0.235872
Name: Value, dtype: float64


In [14]:
# List of categorical columns
categorical_cols = ["Country Name", "Country Code", "Series Name", "Series Code", "Year"]

# Convert categorical columns to string before encoding
df[categorical_cols] = df[categorical_cols].astype(str)

# Apply Ordinal Encoding
encoder = OrdinalEncoder()
df[categorical_cols] = encoder.fit_transform(df[categorical_cols])

print("\nEncoded Categorical Data Preview:\n", df.head())


Encoded Categorical Data Preview:
    Country Name  Country Code  Series Name  Series Code  Year     Value
0           3.0           5.0         59.0         16.0   1.0 -0.235872
1           3.0           5.0         65.0         36.0   1.0 -0.235872
2           3.0           5.0         71.0         31.0   1.0 -0.235872
3           3.0           5.0         68.0         21.0   1.0 -0.235872
4           3.0           5.0         62.0         26.0   1.0 -0.235872


In [15]:
# Display missing values after processing
print("\nMissing Values AFTER Handling:\n", df.isnull().sum())


Missing Values AFTER Handling:
 Country Name    0
Country Code    0
Series Name     0
Series Code     0
Year            0
Value           0
dtype: int64


In [16]:
# Summary of numerical data
print("\n Descriptive Statistics for Numerical Data:")
print(df.describe())

# Summary of categorical columns
categorical_cols = ["Country Name", "Country Code", "Series Name", "Series Code", "Year"]
print("\n Unique Categories in Categorical Columns:")
for col in categorical_cols:
    print(f"{col}: {df[col].nunique()} unique values")


 Descriptive Statistics for Numerical Data:
       Country Name   Country Code    Series Name    Series Code  \
count  166218.00000  166218.000000  166218.000000  166218.000000   
mean      133.49935     132.225481      92.854294      86.419738   
std        76.50603      76.236758      52.174338      54.567136   
min         0.00000       0.000000       0.000000       0.000000   
25%        65.00000      65.000000      53.000000      37.000000   
50%       136.00000     133.000000      88.000000      82.000000   
75%       201.00000     197.000000     137.000000     134.000000   
max       265.00000     265.000000     189.000000     189.000000   

                Year         Value  
count  166218.000000  1.662180e+05  
mean        5.707866 -1.641509e-17  
std         4.276599  1.000003e+00  
min         0.000000 -2.427177e-01  
25%         1.000000 -2.358721e-01  
50%         5.000000 -2.358721e-01  
75%        10.000000 -2.312884e-01  
max        14.000000  9.206024e+00  

 Unique 